## OpenWeatherMap API

In [ ]:
import requests
import asyncio
import aiohttp
import time
from datetime import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

OPENWEATHER_API_KEY = ""



In [18]:
def get_current_season():
    month = datetime.now().month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

def check_temperature_normality(current_temp, city_name, season, historical_data):
    city_season_data = historical_data[
        (historical_data['city'] == city_name) & 
        (historical_data['season'] == season)
    ]
    
    if len(city_season_data) == 0:
        return {
            'status': 'unknown',
            'message': f'Нет исторических данных для {city_name} в сезон {season}'
        }
    
    mean_temp = city_season_data['temperature'].mean()
    std_temp = city_season_data['temperature'].std()
    
    lower_bound = mean_temp - 2 * std_temp
    upper_bound = mean_temp + 2 * std_temp
    
    is_normal = lower_bound <= current_temp <= upper_bound
    
    deviation = current_temp - mean_temp
    deviation_in_std = deviation / std_temp if std_temp > 0 else 0
    
    result = {
        'city': city_name,
        'season': season,
        'current_temp': current_temp,
        'mean_temp': mean_temp,
        'std_temp': std_temp,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'is_normal': is_normal,
        'deviation': deviation,
        'deviation_in_std': deviation_in_std,
        'status': 'normal' if is_normal else 'anomaly'
    }    
    return result



### Синхронный подход

In [ ]:
def get_current_temperature_sync(city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric',
        'lang': 'ru'
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        result = {
            'city': city_name,
            'temperature': data['main']['temp'],
            'feels_like': data['main']['feels_like'],
            'temp_min': data['main']['temp_min'],
            'temp_max': data['main']['temp_max'],
            'pressure': data['main']['pressure'],
            'humidity': data['main']['humidity'],
            'description': data['weather'][0]['description'],
            'wind_speed': data['wind']['speed'],
            'timestamp': datetime.fromtimestamp(data['dt']),
            'success': True,
            'error': None
        }
        return result 
    except requests.exceptions.RequestException as e:
        return {
            'city': city_name,
            'temperature': None,
            'success': False,
            'error': str(e)
        }


def monitor_cities_sync(cities, api_key):
    results = []
    start_time = time.time()
    
    for city in cities:
        print(f"Запрос для {city}", end=' ')
        result = get_current_temperature_sync(city, api_key)
        
        if result['success']:
            print(f"{result['temperature']:.1f}")
        else:
            print(result['error'])
        
        results.append(result)
    
    elapsed_time = time.time() - start_time
    return results, elapsed_time


In [8]:
test_cities = ['Berlin', 'Cairo', 'Dubai', 'Beijing', 'Moscow']
sync_results, sync_time = monitor_cities_sync(test_cities, OPENWEATHER_API_KEY)

print(f"Время выполнения: {sync_time:.3f} секунд")
print(f"Среднее время на город: {sync_time/len(test_cities):.3f} секунд")

sync_df = pd.DataFrame([r for r in sync_results if r['success']])

if len(sync_df) > 0:
    print("\nПолученные данные:")
    display_cols = ['city', 'temperature', 'feels_like', 'humidity', 'description']
    print(sync_df[display_cols].to_string(index=False))


Запрос для Berlin -7.1
Запрос для Cairo 17.4
Запрос для Dubai 23.0
Запрос для Beijing -11.1
Запрос для Moscow -4.7
Время выполнения: 1.001 секунд
Среднее время на город: 0.200 секунд

Полученные данные:
   city  temperature  feels_like  humidity          description
 Berlin        -7.08      -11.63        79 небольшая облачность
  Cairo        17.42       16.55        51                 ясно
  Dubai        22.96       22.59        49                 ясно
Beijing       -11.06      -14.90        20                 ясно
 Moscow        -4.74       -9.95        87             пасмурно


### Асинхронный подход

In [6]:
async def get_current_temperature_async(session, city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric',
        'lang': 'ru'
    }
    
    try:
        async with session.get(base_url, params=params, timeout=10) as response:
            response.raise_for_status()
            data = await response.json()
            
            result = {
                'city': city_name,
                'temperature': data['main']['temp'],
                'feels_like': data['main']['feels_like'],
                'temp_min': data['main']['temp_min'],
                'temp_max': data['main']['temp_max'],
                'pressure': data['main']['pressure'],
                'humidity': data['main']['humidity'],
                'description': data['weather'][0]['description'],
                'wind_speed': data['wind']['speed'],
                'timestamp': datetime.fromtimestamp(data['dt']),
                'success': True,
                'error': None
            }
            return result
    except Exception as e:
        return {
            'city': city_name,
            'temperature': None,
            'success': False,
            'error': str(e)
        }


async def monitor_cities_async(cities, api_key):
    start_time = time.time()
    
    async with aiohttp.ClientSession() as session:
        tasks = [get_current_temperature_async(session, city, api_key) for city in cities]
        results = await asyncio.gather(*tasks)
    
    elapsed_time = time.time() - start_time
    
    for result in results:
        if result['success']:
            print(f"{result['city']}: {result['temperature']:.1f}")
        else:
            print(f"{result['city']}: {result['error']}")
    
    return results, elapsed_time


In [ ]:
async_results, async_time = await monitor_cities_async(test_cities, OPENWEATHER_API_KEY)

print(f"Время выполнения: {async_time:.3f} секунд")
print(f"Среднее время на город: {async_time/len(test_cities):.3f} секунд")

async_df = pd.DataFrame([r for r in async_results if r['success']])

if len(async_df) > 0:
    print("\nПолученные данные:")
    display_cols = ['city', 'temperature', 'feels_like', 'humidity', 'description']
    print(async_df[display_cols].to_string(index=False))


Berlin: -7.1
Cairo: 17.4
Dubai: 23.0
Beijing: -11.1
Moscow: -4.7
Время выполнения: 0.206 секунд
Среднее время на город: 0.041 секунд

Полученные данные:
   city  temperature  feels_like  humidity          description
 Berlin        -7.08      -11.63        79 небольшая облачность
  Cairo        17.42       16.55        51                 ясно
  Dubai        22.96       22.59        49                 ясно
Beijing       -11.06      -14.90        20                 ясно
 Moscow        -4.74       -9.95        87             пасмурно


### Аномалии

In [23]:

historical_data = pd.read_csv('temperature_data.csv')
historical_data['timestamp'] = pd.to_datetime(historical_data['timestamp'])
historical_data['season'] = historical_data['season'].str.lower().str.strip()

current_season = get_current_season()
print(f"\nТекущий сезон: {current_season}")

city_mapping = {
    'Berlin': 'Берлин',
    'Cairo': 'Каир',
    'Dubai': 'Дубай',
    'Beijing': 'Пекин',
    'Moscow': 'Москва'
}

city_mapping_reverse = {v: k for k, v in city_mapping.items()}
analysis_results = []
for result in async_results:
    if not result['success']:
        continue
    
    city_eng = result['city']
    city_rus = city_mapping.get(city_eng, city_eng)
    current_temp = result['temperature']
    
    city_in_data = None
    if city_rus in historical_data['city'].values:
        city_in_data = city_rus
    elif city_eng in historical_data['city'].values:
        city_in_data = city_eng
    
    city_season_data = historical_data[
        (historical_data['city'] == city_in_data) & 
        (historical_data['season'] == current_season)
    ]
    
    mean_temp = city_season_data['temperature'].mean()
    std_temp = city_season_data['temperature'].std()
    
    lower_bound = mean_temp - 2 * std_temp
    upper_bound = mean_temp + 2 * std_temp
    
    is_normal = lower_bound <= current_temp <= upper_bound
    
    deviation = current_temp - mean_temp
    deviation_in_std = deviation / std_temp if std_temp > 0 else 0
    
    analysis = {
        'city': city_rus,
        'city_eng': city_eng,
        'season': current_season,
        'current_temp': current_temp,
        'mean_temp': mean_temp,
        'std_temp': std_temp,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'is_normal': is_normal,
        'deviation': deviation,
        'deviation_in_std': deviation_in_std,
        'description': result['description'],
        'timestamp': result['timestamp']
    }
    
    analysis_results.append(analysis)




Текущий сезон: winter


In [24]:
normal_cities = []
anomaly_cities = []

for analysis in analysis_results:
    city = analysis['city']
    current = analysis['current_temp']
    mean = analysis['mean_temp']
    std = analysis['std_temp']
    lower = analysis['lower_bound']
    upper = analysis['upper_bound']
    deviation = analysis['deviation']
    dev_std = analysis['deviation_in_std']
    is_normal = analysis['is_normal']
    season_ru = analysis['season']
    
    print(f"\n{city} ({analysis['city_eng']})")
    print(f"Описание: {analysis['description']}")
    print(f"Текущая температура: {current:.1f}")
    print(f"Историческое среднее ({season_ru}): {mean:.1f}°(std={std:.1f})")
    print(f"Диапазон нормы: [{lower:.1f}, {upper:.1f}]")
    print(f"Отклонение от среднего: {deviation:+.1f} (std={dev_std:+.2f})")
    
    if is_normal:
        print(f"Статус: НОРМАЛЬНАЯ ТЕМПЕРАТУРА")
        normal_cities.append(city)
    else:
        if current > upper:
            anomaly_type = "аномально ВЫСОКАЯ"
        else:
            anomaly_type = "аномально НИЗКАЯ"
        print(f"Статус: АНОМАЛИЯ {anomaly_type}")
        anomaly_cities.append(city)
    




Берлин (Berlin)
Описание: небольшая облачность
Текущая температура: -7.1
Историческое среднее (winter): 0.3°(std=5.1)
Диапазон нормы: [-9.9, 10.5]
Отклонение от среднего: -7.4 (std=-1.45)
Статус: НОРМАЛЬНАЯ ТЕМПЕРАТУРА

Каир (Cairo)
Описание: ясно
Текущая температура: 17.4
Историческое среднее (winter): 15.0°(std=5.0)
Диапазон нормы: [4.9, 25.1]
Отклонение от среднего: +2.4 (std=+0.48)
Статус: НОРМАЛЬНАЯ ТЕМПЕРАТУРА

Дубай (Dubai)
Описание: ясно
Текущая температура: 23.0
Историческое среднее (winter): 19.6°(std=5.0)
Диапазон нормы: [9.6, 29.6]
Отклонение от среднего: +3.4 (std=+0.67)
Статус: НОРМАЛЬНАЯ ТЕМПЕРАТУРА

Пекин (Beijing)
Описание: ясно
Текущая температура: -11.1
Историческое среднее (winter): -2.1°(std=4.8)
Диапазон нормы: [-11.8, 7.5]
Отклонение от среднего: -8.9 (std=-1.85)
Статус: НОРМАЛЬНАЯ ТЕМПЕРАТУРА

Москва (Moscow)
Описание: пасмурно
Текущая температура: -4.7
Историческое среднее (winter): -10.2°(std=5.2)
Диапазон нормы: [-20.5, 0.1]
Отклонение от среднего: +5.4 (std